# Function calls (a.k.a. tool use) in Guidance

This notebook demonstrates how to get LLMs to call external tools when needed. Tools use can be implementated many ways, because there are many possible ways to design prompts that then produce outputs that can be parsed to trigger external tool calls. You can create and parse of all this syntax yourself, but Guidance also has special support commands for this that align with both the way LLMs are actually executed and with popular APIs like OpenAI. Using this syntax will also help ensure you align your prompts with any fine-tuning the LLM may have undergone for tool use (assuming the corresponding LLM object in Guidance has support built in).

## OpenAI Chat Models

Tool use in Guidance is designed to align with how the model actually processes the text you give it. This means you give the model the actual function definition text the model sees, and you watch for the text generated by the model when it wants to make a function call. While the OpenAI Chat API abstracts away all these details, Guidance re-exposes them so you can interact with OpenAI models in the same way you would interact with any other model.

So in the examples below you will see text going into the model's system prompt, and function calls coming out of the model as though you were watching the raw model output inside the `assistant` role. But behind the scenes the `guidance.llms.OpenAI` class translates this text into the corrresponding API calls. Note that the text that Guidance puts into the system prompt follows the TypeScript format that ChatGPT claims to expect on the backend, so you are seeing what things look like to the LLM itself (note you can just ask ChatGPT what it expects in order to get this format).

### Define the tool(s) we want to use

Here we use the same mock tool that is used in the OpenAI docs, a weather service function.

In [1]:
# define a tool we would like the model to use
import json
def get_current_weather(location, unit="fahrenheit"):
    """ Get the current weather in a given location.
    
    Parameters
    ----------
    location : string
        The city and state, e.g. San Francisco, CA
    unit : "celsius" or "fahrenheit"
    """
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

### Define a guidance program that uses the tool(s)

To get the LLM to use tools when it needs it you need to first specify which tools it can use, then you need to watch for when the LLM wants to use a tool.

**Function definition** can be done in many ways, but if you want to align with how the model was fine tuned you can use the special `llm.tool_def` guidance program. This program is defined by the LLM running your program and it will convert a list of function definitions into a format that the model understands (where the function definitions are the same as the OpenAI API expects). For OpenAI models the text generated by `llm.tool_def` is TypeScript type definitions and belongs at the end of the `system` message.

**Call detection** can be done manually by setting the `stop` or `stop_regex` parameters of the `gen` command to something that signifies that the LLM is making a function call. But a cleaner way is to use the `function_call="auto"` parameter. This will get passed directly to the LLM object so that it can set the appropriate `stop_regex` parameter or API parameter. If your LLM defines the `llm.extract_function_call` method then you can treat the returned text just like a function, so calling the string will result in calling the tool call embedded in that string. This makes it easy to work with tool call outputs in the same way you work with other outputs from the LLM.

Below is an example that puts all this together for the OpenAI Chat API:




In [2]:
import guidance

# define the chat model we want to use (must be a recent model supporting function calls)
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo-0613", caching=False)

# define a guidance program that uses tools
program = guidance("""
{{~#system~}}
You are a helpful assistant.
{{>@tool_def functions=functions}}
{{~/system~}}

{{~#user~}}
Get the current weather in New York City.
{{~/user~}}

{{~#each range(10)~}}
    {{~#assistant~}}
    {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
    {{~/assistant~}}

    {{#if callable(answer)}}
        {{~#function name=answer.function_name~}}
        {{answer()}}
        {{~/function~}}
    {{else}}{{break}}{{/if}}
{{~/each~}}""")

Note that in the program above we make a maximum of 10 consecutive function calls. Once the model generates text that does not include a function call we break out and leave the text from that final answer in the `answer` variable.

### Calling the guidance program

To call the program above we need to pass in a function definition and the actual function to call.

In [4]:
# call the program, passing in the function definition we want to use as JSON
executed_program = program(functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        }
    }
], get_current_weather=get_current_weather)

Traceback (most recent call last):
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 113, in run
    await self.visit(self.parse_tree, VariableStack([self.program._variables], self))
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 551, in visit
    visited_children.append(await self.visit(child, variable_stack, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 516, in visit
    command_output = await command_function(*positional_args, **named_args)
  File "/home/sclundbe/projects/guidance/guidance/library/_system.py", line 13, in system
    return await role(role_name="system", hidden=hidden, _parser_context=_parser_context, **kwargs)
  File "/home/sclundbe/projects/guidance/guidance/library/_role.py", line 17, in role
    new_content += await parser.visit(
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py

AssertionError: Expected else statement

In [36]:
executed_program["answer"]

'The current weather in New York City is 72°F and it is sunny and windy.'

### Factoring out the loop calling code

There is a non-trivial amount of logic and syntax requires to create a loop of function calls. We can make this easer by factoring out that loop into its own guidance program:

In [3]:
# this is a reusabe component for calling functions as intermediate steps in a generation
chat_tool_gen = guidance("""{{~#each range(max_calls)~}}
    {{~#assistant~}}
    {{gen args[0] temperature=temperature max_tokens=max_tokens function_call=function_call}}
    {{~/assistant~}}

    {{#if callable(answer)}}
        {{~#function name=answer.function_name~}}
        {{answer()}}
        {{~/function~}}
    {{else}}{{break}}{{/if}}
{{~/each~}}""", max_calls=20, function_call="auto", max_tokens_per_chunk=500, temperature=0.0)

# define a guidance program that uses chat_tool_gen
program2 = guidance("""
{{~#system~}}
You are a helpful assistant.
{{>@tool_def functions=functions}}
{{~/system~}}

{{~#user~}}
Get the current weather in New York City.
{{~/user~}}

{{>chat_tool_gen 'answer'}}""", chat_tool_gen=chat_tool_gen)

executed_program2 = program2(functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        }
    }
], get_current_weather=get_current_weather)

Traceback (most recent call last):
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 113, in run
    await self.visit(self.parse_tree, VariableStack([self.program._variables], self))
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 551, in visit
    visited_children.append(await self.visit(child, variable_stack, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py", line 516, in visit
    command_output = await command_function(*positional_args, **named_args)
  File "/home/sclundbe/projects/guidance/guidance/library/_system.py", line 13, in system
    return await role(role_name="system", hidden=hidden, _parser_context=_parser_context, **kwargs)
  File "/home/sclundbe/projects/guidance/guidance/library/_role.py", line 17, in role
    new_content += await parser.visit(
  File "/home/sclundbe/projects/guidance/guidance/_program_executor.py

AssertionError: Expected else statement

### Calling the function outside of Guidance [IN PROGRESS, NOT YET WORKING]

In the example above the function call was made during the guidance program, but we can also pause the program's execution whenever we want to make a function call, and then make that call outside of Guidance. This is useful if you don't want Guidance to own the function calling part of your program.

In [39]:
# define a guidance program that pauses we when a function call is made
await_program = guidance("""
{{~#system~}}
You are a helpful assistant.
{{>@tool_def functions=functions}}
{{~/system~}}

{{~#user~}}
Get the current weather in New York City.
{{~/user~}}

{{~#each range(10)~}}
    {{~#assistant~}}
    {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
    {{~/assistant~}}

    {{#if callable(answer)}}
        {{set 'answer' await('call_result')}}
        {{~#function name=answer.function_name~}}
        {{answer}}
        {{~/function~}}
    {{else}}{{break}}{{/if}}
{{~/each~}}""")

# call the program, passing in the function definition we want to use as JSON
executed_await_program = await_program(functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        }
    }
], get_current_weather=get_current_weather)

Stop program system You are a helpful assistant.

# Tools

<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='{{#if len(functions) > 0~}}
## functions

namespace functions {

{{#each functions item_name="function"~}}
// {{function.description}}
type {{function.name}} = (_: {
{{~#each function.parameters.properties}}
{{#if contains(this, "description")}}// {{this.description}}
{{/if~}}
{{@key}}{{#unless contains(function.parameters.required, @key)}}?{{/unless}}: {{#if contains(this, "enum")}}{{#each this.enum}}"{{this}}"{{#unless @last}} | {{/unless}}{{/each}}{{else}}{{this.type}}{{/if}}{{#unless @last}},{{/unless}}
{{~/each}}
}) => any;

{{/each~}}
} // namespace functions
{{~/if~}}'>## functions

namespace functions {

<span style='opacity: 1.0; display: inline; background-color: rgba(165, 165, 165, 0.1);' title='{{#each functions item_name="function"~}}
// {{function.description}}
type {{function.name}} = (_: {
{{~#each function.parameters.properties}}
{{#if contains(this, "description")}}// {{this.description}}
{{/if~}}
{{@key}}{{#unless contains(function.parameters.required, @key)}}?{{/unless}}: {{#if contains(this, "enum")}}{{#each this.enum}}"{{this}}"{{#unless @last}} | {{/unless}}{{/each}}{{else}}{{this.type}}{{/if}}{{#unless @last}},{{/unless}}
{{~/each}}
}) => any;

{{/each~}}
'>// Get the current weather in a given location 
type get_current_weather = (_: { 
 // The city and state, e.g. San Francisco, CA 
 location : string , 
 unit ? : " celsius " | " fahrenheit " 
}) => any;

 } // namespace functions user Get the current weather in New York City. assistant 

```typescript
functions.get_current_weather({
 "location": "New York City"
})``` 
 {{set 'answer' await('call_result')}}{{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen 'answer' temperature=1.0 max_tokens=50 function_call="auto"}}
 {{~/assistant~}}

 {{#if callable(answer)}}
 {{set 'answer' await('call_result')}}
 {{~#function name=answer.function_name~}}
 {{answer}}
 {{~/function~}}
 {{else}}{{break}}{{/if}}{{~#assistant~}}
 {{gen '

## Open Source Falcon model [IN PROGRESS, NOT YET WORKING]

Here we run the same examples as before, but with the Falcon model instead to show how function calls work with other models. Note that the Falcon model does not have any special fine-tuned support for function calls, so we have to provide much more detail in the tool definition.

In [ ]:
chat_tool_gen = guidance("""{{~#each range(max_calls)~}}
    {{~#assistant~}}
    {{gen args[0] temperature=temperature max_tokens=max_tokens function_call=function_call}}
    {{~/assistant~}}

    {{#if callable(answer)}}
        {{~#function name=answer.function_name~}}
        {{answer()}}
        {{~/function~}}
    {{else}}{{break}}{{/if}}
{{~/each~}}""", max_calls=20, function_call="auto", max_tokens_per_chunk=50, temperature=0.0)